In [17]:
import pandas as pd
import random



# Function to sort FEL chronologically
def fel_sort(FEL):
    i = len(FEL)-1
    while i > 0:
        for j in range(i):
            if FEL[j][0] == 'E':
                FEL[j],FEL[j+1] = FEL[j+1],FEL[j]
        i -= 1
    i = len(FEL)-1
    while i > 0:
        for j in range(i):
            if FEL[j][1] > FEL[j+1][1]:
                FEL[j],FEL[j+1] = FEL[j+1],FEL[j]
        i -= 1
    return FEL


# Function to generate Loading Time
def load_time( ):
    r = random.random()
    if r>=0.0 and r<=0.3:
        load_time = 5
    elif r>0.3 and r<=0.8:
        load_time = 10
    elif r>0.8 and r<=1.0:
        load_time = 15
    return load_time


# Function to generate Weighing Time
def weighing_time( ):
    r = random.random()
    if r>=0.0 and r<=0.7:
        weighing_time = 12
    elif r>0.7 and r<=1.0:
        weighing_time = 16
    return weighing_time


# Function to generate Travel Time
def travel_time( ):
    r = random.random()
    if r>=0.0 and r<=0.4:
        travel_time = 40
    elif r>0.4 and r<=0.7:
        travel_time = 60
    elif r>0.7 and r<=0.9:
        travel_time = 80
    elif r>0.9 and r<=1.0:
        travel_time = 100    
    return travel_time


# Function to remove a Dump Truck from the loading queue
def del_loading(lqueue,dt):
    lqueue.remove(dt)
    return lqueue


# Function to add a Dump Truck to the loading queue
def add_loading(lqueue,dt):
    lqueue.append(dt)
    return lqueue


# Function to add a Dump Truck into the weighing queue
def add_weigh(wqueue,dt):
    wqueue.append(dt)
    return wqueue


# Function to remove a Dump Truck from the weighing queue
def del_weigh(wqueue):
    wqueue.remove(wqueue[0])
    return wqueue


CLOCK = [ ]                 # Clock
L = [ ]                     # No. of dump trucks being loaded
LQ = [ ]                    # No. of dump trucks in loading queue
W = [ ]                     # No. of dump trucks being weighed
WQ = [ ]                    # No. of dump trucks in weighing queue
FEL_LIST = [ ]              # Future Event List
LOADING = [ ]               # List of dump trucks in the loading system
WEIGHING = [ ]              # List of dump trucks in the weighing system
BL = [ ]                    # Total busy time of both loaders
BS = [ ]                    # Total busy time of the scale


# Initial conditions
clk = 0
FEL = [('E',2000)]
l = 2
lq = 3
w = 1
wq = 0
lqueue = ['DT2', 'DT3', 'DT4', 'DT5', 'DT6']
wqueue = ['DT1']
bl = 0
bs = 0



# Simulation
while clk < 2000:
    if clk == 0:
        # At CLOCK=0, DT2 and DT3 are being loaded while DT4,DT5 and DT6 waits in loading queue and DT1 is being 
        # weighed. Therefore, loading times are generated for DT2 and DT3 and weighing time is generated for DT1.
        # The corresponding End-Loading and End-Weighing Events are scheduled and added to FEL.
        lt = load_time()
        EL = clk + lt
        FEL.append(('EL',EL,'DT2'))
        lt = load_time()
        EL = clk + lt
        FEL.append(('EL',EL,'DT3'))
        wt = weighing_time()
        EW = clk + wt
        FEL.append(('EW',EW,'DT1'))
        FEL = fel_sort(FEL)
    elif clk != 0:
        imminent_event_notice = FEL[0]
        FEL.remove(FEL[0])
        if imminent_event_notice[0] == 'EL':
            # End-Loading Event Simulation
            dt_exit_load = imminent_event_notice[2]
            if lq > 0:
                lq -= 1
                dt_to_load = lqueue[2]
                lt = load_time()
                EL = clk + lt
                FEL.append(('EL',EL,dt_to_load))
                FEL = fel_sort(FEL)
            else:
                l -= 1
            # Removing the loaded truck from the list of trucks in the loading system
            lqueue = del_loading(lqueue,dt_exit_load)
            # Adding the loaded truck to the list of trucks in the weighing system
            if w == 1:
                wq += 1
            else:
                w = 1
                wt = weighing_time()
                EW = clk + wt
                FEL.append(('EW',EW,dt_exit_load))
                FEL = fel_sort(FEL)
            wqueue = add_weigh(wqueue,dt_exit_load)
        elif imminent_event_notice[0] == 'EW':
            # End-Weighing Event Simulation
            dt_exit_weigh = imminent_event_notice[2]
            if wq > 0:
                wq -= 1
                dt_to_weigh = wqueue[1]
                wt = weighing_time()
                EW = clk + wt
                FEL.append(('EW',EW,dt_to_weigh))
                FEL = fel_sort(FEL)
            else:
                w = 0
            # Remove the weighed truck from the list of trucks in the weighing system
            wqueue = del_weigh(wqueue)
            tt = travel_time()
            ALQ = clk + tt
            FEL.append(('ALQ',ALQ,dt_exit_weigh))
            FEL = fel_sort(FEL)
        elif imminent_event_notice[0] == 'ALQ':
            # Arriving at Loading Simulation
            dt_to_arrive = imminent_event_notice[2]
            if l == 2:
                lq += 1
            else:
                l += 1
                lt = load_time()
                EL = clk + lt
                FEL.append(('EL',EL,dt_to_arrive))
                FEL = fel_sort(FEL)
            # Adding the arrived truck to the list of trucks in the loading system 
            lqueue = add_loading(lqueue,dt_to_arrive)
    L.append(l)
    LQ.append(lq)
    W.append(w)
    WQ.append(wq)
    temp = list(FEL)
    FEL_LIST.append(temp)
    temp = list(lqueue)
    LOADING.append(temp)
    temp = list(wqueue)
    WEIGHING.append(temp)
    CLOCK.append(clk)
    clk = FEL[0][1]



# Calculating total busy time of both loaders
BL.append(bl)
for i in range(1,len(CLOCK)):
    bl += L[i-1]*(CLOCK[i]-CLOCK[i-1])
    BL.append(bl)
    
# Calculating total busy time of the scale
BS.append(bs)
for i in range(1,len(CLOCK)):
    bs += W[i-1]*(CLOCK[i]-CLOCK[i-1])
    BS.append(bs)
    
# Calculating loader and scale utilizations
load_util = (BL[-1]/2)/CLOCK[-1]
scale_util = BS[-1]/CLOCK[-1]
print(f'Average loader utilization = {round(load_util,2)}')
print(f'Average scale utilization = {round(scale_util,2)}')



for i in LOADING:
    i.reverse()
for i in WEIGHING:
    i.reverse()

for i in range(len(FEL_LIST)):
    s = str(FEL_LIST[i])
    st = s[1:-1]
    FEL_LIST[i] = st
for i in range(len(LOADING)):
    s = str(LOADING[i])
    st = s[1:-1]
    LOADING[i] = st
for i in range(len(WEIGHING)):
    s = str(WEIGHING[i])
    st = s[1:-1]
    WEIGHING[i] = st



# Simulation Table
queue = pd.DataFrame( )
queue['CLOCK'] = CLOCK
queue['LQ(t)'] = LQ
queue['L(t)'] = L
queue['WQ(t)'] = WQ
queue['W(t)'] = W
queue['LOADING'] = LOADING
queue['WEIGHING'] = WEIGHING
queue['FUTURE EVENT LIST'] = FEL_LIST
queue['BL'] = BL
queue['BS'] = BS

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 50)
display(queue)


Average loader utilization = 0.3
Average scale utilization = 0.86


,CLOCK,LQ(t),L(t),WQ(t),W(t),LOADING,WEIGHING,FUTURE EVENT LIST,BL,BS
0,0,3,2,0,1,"'DT6', 'DT5', 'DT4', 'DT3', 'DT2'",'DT1',"('EL', 5, 'DT2'), ('EL', 10, 'DT3'), ('EW', 12...",0,0
1,5,2,2,1,1,"'DT6', 'DT5', 'DT4', 'DT3'","'DT2', 'DT1'","('EL', 10, 'DT3'), ('EW', 12, 'DT1'), ('EL', 1...",10,5
2,10,1,2,2,1,"'DT6', 'DT5', 'DT4'","'DT3', 'DT2', 'DT1'","('EW', 12, 'DT1'), ('EL', 15, 'DT4'), ('EL', 2...",20,10
3,12,1,2,1,1,"'DT6', 'DT5', 'DT4'","'DT3', 'DT2'","('EL', 15, 'DT4'), ('EL', 20, 'DT5'), ('EW', 2...",24,12
4,15,0,2,2,1,"'DT6', 'DT5'","'DT4', 'DT3', 'DT2'","('EL', 20, 'DT5'), ('EW', 24, 'DT2'), ('EL', 2...",30,15
5,20,0,1,3,1,'DT6',"'DT5', 'DT4', 'DT3', 'DT2'","('EW', 24, 'DT2'), ('EL', 25, 'DT6'), ('ALQ', ...",40,20
6,24,0,1,2,1,'DT6',"'DT5', 'DT4', 'DT3'","('EL', 25, 'DT6'), ('EW', 40, 'DT3'), ('ALQ', ...",44,24
7,25,0,0,3,1,,"'DT6', 'DT5', 'DT4', 'DT3'","('EW', 40, 'DT3'), ('ALQ', 84, 'DT2'), ('ALQ',...",45,25
8,40,0,0,2,1,,"'DT6', 'DT5', 'DT4'","('EW', 56, 'DT4'), ('ALQ', 80, 'DT3'), ('ALQ',...",45,40
9,56,0,0,1,1,,"'DT6', 'DT5'","('EW', 68, 'DT5'), ('ALQ', 80, 'DT3'), ('ALQ',...",45,56
